# MUSA620 Final Project

## Set-up and Data Cleaning/Exploration

### Load in Packages and Data

In [ ]:
#Load in necessary packages 
import datetime
import pandas as pd
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cycler
import seaborn as sns
import altair as alt
from vega_datasets import data
import hvplot.pandas
import numpy as np

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

import os, numpy as np, pandas as pd, cartopy.crs as ccrs, bokeh
import holoviews as hv, geoviews as gv, datashader as ds

from colorcet import bmy
from holoviews.util import Dynamic
from holoviews.operation.datashader import rasterize, datashade

from holoviews import opts, streams
from holoviews.plotting.links import DataLink

import param as pm
import panel as pn
from colorcet import cm

In [ ]:
# Enable Altair and Holoviews rendering in the notebook
alt.renderers.enable('notebook')
hv.extension('bokeh')

### Initial Data Preparation/Exploration of Pipeline Data



In [ ]:
#Load in data
pipeline_raw_data = pd.read_csv('database.csv')
#pipeline_raw_data

In [ ]:
#Remove 2017
pipeline_df = pipeline_raw_data[pipeline_raw_data["Accident Year"] != 2017]

In [ ]:
pipeline_df.columns

In [ ]:
pipeline_by_year_df = pipeline_df.groupby(["Accident Year"]).sum().reset_index()
#pipeline_by_year_df

In [ ]:
pipeline_by_state_df = pipeline_df.groupby(["Accident State"]).sum().reset_index()
#pipeline_by_state_df

### Visualizations Using Just Pipeline Data

## How do the number of pipeline incidents vary by year (with breakdown by cause)?

Bar chart showing number of incidents per year with a breakdown by type.

In [ ]:
incidents_by_year = pipeline_df.groupby(["Accident Year", "Cause Category"])['Report Number'].count()
#incidents_by_year

In [ ]:
incidents_by_year_plot = incidents_by_year.hvplot(kind='bar'
                         , xlabel='Year'
                         , ylabel='Number of Incidents'
                         , ylim=(0, 800)
                         , rot=90
                         , width=400
                         , height = 600                         
                         , stacked=True
                         , title = "Number of Incidents by Year and Cause"
                         , cmap=[
                             '#abcca9',
                             '#a7bd86',
                             '#acab62',
                             '#b69643',
                             '#c27d2e',
                             '#cd5e2a',
                             '#d43535']).opts(xrotation=45, legend_position='top_left', bgcolor='white', fontsize={'title': 12, 'labels': 10, 'xticks': 10, 'yticks': 10})

incidents_by_year_plot

## How do the number of incidents compare across states (with breakdown by year)?

A heatmap showing the number of incidents per state by year. This section could also include a table allowing the user to view the states alphabetically or by number of incidents or a map color coded by number of incidents.

In [ ]:
state_count = pipeline_df.groupby(["Accident State"])['Report Number'].count().reset_index()
#state_count

In [ ]:
#Create Table of the Number of Incidents Over This Time Period by Number of Incidents
number_of_incidents = state_count['Report Number']
states = state_count['Accident State']
state_table = hv.Table({"State": states, 'Number of Incidents': number_of_incidents}, ['State', 'Number of Incidents']).opts(height= 300
                                                                                                                      , width = 300)                                                                                                                    
state_table

In [ ]:
#Get all state and year combos
import itertools

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "PR"]

years = ["2010", "2011", "2012", "2013", "2014", "2015", "2016"]

a = [states, years]

combos = list(itertools.product(*a))

state = [x[0] for x in combos]
year = [x[1] for x in combos]

#print(combos)
#len(state)
#len(year)

In [ ]:
#Heatmap Data Preparation

#Get All States and Years (for data Cleaning)
data = {'Accident State': state, 'Accident Year': year}
  
# Create DataFrame 
states_years = pd.DataFrame(data) 

#Create State Year Count
state_year_count = pipeline_df.groupby(["Accident Year", "Accident State"])['Report Number'].count().reset_index()

state_year_count['Accident Year'] = state_year_count['Accident Year'].apply(lambda x: str(x))

#Make the state-year combos with no reports equal to zero
state_year_count = pd.merge(states_years, state_year_count, on = ["Accident State", "Accident Year"], how = "outer")

#state_year_count[state_year_count['Accident State']=="OR"]

state_year_count = state_year_count.fillna(0)


In [ ]:
#Create Heatmap
dataset = hv.Dataset(state_year_count, vdims=('Report Number','Number of Incidents'))

incident_heatmap = hv.HeatMap(dataset.aggregate(["Accident Year", "Accident State"], np.sum),label='Number of Incidents by Year and State').opts(height=600
                                                                                                                      , width = 400
                                                                                                                      , colorbar=True
                                                                                                                      , tools=['hover']
                                                                                                                      , logz = True
                                                                                                                      , cmap = 'Oranges'
                                                                                                                      , xrotation=45
                                                                                                                      , xlabel = "Year"
                                                                                                                      , ylabel = "State")

incident_heatmap

## Where are the locations of the incidents and how to this vary by year?

A map showing the locations of the pipeline incidents with ability to compare by year?

In [ ]:
from bokeh.models import HoverTool
hover_locations = HoverTool(tooltips=[('Accident Year', '$Accident Year')])

pipeline_location_data = pipeline_df.copy()

pipeline_location_data['Accident Year'] = pipeline_location_data['Accident Year'].apply(lambda x: str(x))

pipeline_location_map = pipeline_location_data.hvplot.points('Accident Longitude', 'Accident Latitude', title = "", geo=True, alpha=0.2, legend = False,
                       xlim=(-180, -30), ylim=(0, 72), tiles='CartoLight', frame_width = 600, c='#d43535', groupby='Accident Year', tools=[hover_locations]) #color='#d43535', c='Accident Year' 


pipeline_location_map.opts(xaxis=None, yaxis=None)

In [ ]:
from bokeh.models import HoverTool
hover_locations = HoverTool(tooltips=[('Accident Year', '$Accident Year')])

pipeline_location_data = pipeline_df.copy()

pipeline_location_data['Accident Year'] = pipeline_location_data['Accident Year'].apply(lambda x: str(x))

pipeline_location_map_2 = pipeline_location_data.hvplot.points('Accident Longitude', 'Accident Latitude', title = "", geo=True, alpha=0.2, legend = True,
                       xlim=(-180, -30), ylim=(0, 72), tiles='CartoLight', frame_width = 600, c='#61594e', tools=[hover_locations]) #color='#d43535', c='Accident Year' 


pipeline_location_map_2.opts(xaxis=None, yaxis=None)

## How do the number of incidents compare across companies?

Similar to above, a table showing the number of incidents per company either overall or by year. Again, the table should allow the user to view the states alphabetically or by number of incidents.

In [ ]:
company_count = pipeline_df.groupby(["Operator Name"])['Report Number'].count().reset_index()
#company_count

#Create Table of the Number of Incidents Over This Time Period by Number of Incidents
number_of_incidents = company_count['Report Number']
company = company_count['Operator Name']
operator_table = hv.Table({"Operator Name": company, 'Number of Incidents': number_of_incidents}, ['Operator Name', 'Number of Incidents']).opts(height= 300, width = 600, title = "Number of Incidents (2010-2016) by Operator")                                                                                                                    
operator_table


In [ ]:
top_15_offenders = company_count.nlargest(15, 'Report Number')

top_15_offenders

## Final Visualizations (For Reference)

In [ ]:
#finalized plots
#incidents_by_year_plot + incident_heatmap + table
#layout = hv.Layout(incidents_by_year_plot + incident_heatmap + operator_table).cols(2)
#layout
#hvplot.show(layout)
#hvplot.show(pipeline_location_map)

## Create Application

In [ ]:
# The app's title, defined as an h2 HTML elemtn
title = pn.pane.HTML(
    "<h2>Oil Pipeline Spills and Leakages in the United States (2010-2016)</h2>",
    style={"width": "800px", "text-align": "center"},
)

# Description of data included
description = pn.pane.HTML(
    "Map and other visualizations show pipeline incidents from 2010 to 2016 in the United States as reported by PHMSA, the Pipeline and Hazardous Materials Safety Administration." ,
    style={"width": "800px", "text-align": "center"},
)


# The title of the barplot and heatmap
state_title = pn.pane.HTML(
    "<h4>How Do the Number of Incidents Vary by Year and Location?</h4>",
    style={"width": "800px", "text-align": "center"},
)

# The title of the table
operator_title = pn.pane.HTML(
    "<h4>How Do the Number of Incidents Vary by Operator?</h4>",
    style={"width": "800px", "text-align": "center"},
)

# The title of the table
census_title = pn.pane.HTML(
    "<h4>What are the Demographic Trends (by County) Associated with the Pipeline Locations?</h4>",
    style={"width": "800px", "text-align": "center"},
)

In [ ]:
# Layout the dashboard
panel = pn.Column(
    pn.Row(title),
    pn.Row(description),
    pn.Row(pipeline_location_map), #change this to remove "_2" if want original plot
    pn.Row(state_title),
    pn.Row(incidents_by_year_plot, incident_heatmap),
    pn.Row(operator_title),
    pn.Row(pn.Spacer(width=25), operator_table),
    #pn.Row(census_title),
    #pn.Row(demographic_pipeline_location_map),
    width=1200,
    align="center",
)

In [ ]:
panel.servable()

In [ ]:
from bokeh.resources import INLINE
panel.save('test_2.html')

# Additional Visualizations Not Including in Final Report

**How do the number of overall deaths, injuries, and property damages related to spills vary by year and by state?**

Three separate bar charts side-by-side showing the number of deaths, injuries, and property damages per year and by state.

Not including the below few plots because the injury, fatality, costs seem to have some data quality issues.

In [ ]:
costs_injuries_df = pipeline_by_year_df.melt(id_vars=["Accident Year"], value_vars=["All Injuries", "All Fatalities", "All Costs"], var_name="incident_details")
#costs_injuries_df

In [ ]:
costs_injuries_chart = costs_injuries_df.hvplot(x='Accident Year', 
                                 groupby='incident_details', 
                                 width=400, 
                                 dynamic=False, kind='bar')

costs_injuries_chart 

In [ ]:
costs = costs_injuries_chart ['All Costs'].relabel('All Costs')
injuries = costs_injuries_chart ['All Injuries'].relabel('Injuries') 
fatalities = costs_injuries_chart ['All Fatalities'].relabel('Fatalities')

combined = injuries + fatalities    
combined

#However, in the initial dataset this field seems like maybe not that accurate, since blank for most rows - maybe not required to report?

In [ ]:
fatalities = pipeline_by_state_df['All Fatalities']
injuries = pipeline_by_state_df['All Injuries']
costs = pipeline_by_state_df['All Costs']
states = pipeline_by_state_df['Accident State']

table = hv.Table({'State': states, 'Fatalities': fatalities, 'Injuries': injuries, 'Costs': costs},['State', 'Fatalities', 'Injuries', 'Costs'], groupby="Accident Year")

table

### Other (Secondary) Visualization Ideas

- Map color coded by the number of incidents, either over all years or with a drop down by year
- Visualization with a dropdown of costs and can select different types of costs but also the overall costs
- In map where have the locations of all of the pipeline incidents, see if there is a way where there is a table next to it and you can select the pipeline incident and it will be highlighted on the table with information about location, operator name, fatalities, costs, cause, etc. (see small example below)

**What are the geographical locations of the pipeline spills and how are the concentration of spills related to features such as company involved, number of deaths, monetary costs, etc.?**

A map with a point for each incident color coded by, for example, company involved. I would like to make this plot interactive by allowing the user, for example, to select a particular company and just see incidents related to that company. I think a heatmap could also work for this question.